## Deliverable 2. Create a Customer Travel Destinations Map.

In [9]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [10]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,80.56,89,82,5.99,broken clouds
1,1,East London,ZA,-33.0153,27.9116,61.74,82,0,8.05,clear sky
2,2,Mazagao,BR,-0.1150,-51.2894,89.65,70,46,4.59,scattered clouds
3,3,Rikitea,PF,-23.1203,-134.9692,73.08,70,26,9.51,scattered clouds
4,4,Ushuaia,AR,-54.8000,-68.3000,40.62,81,0,4.61,clear sky


In [11]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [12]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Puerto Ayora,EC,-0.7393,-90.3518,80.56,89,82,5.99,broken clouds
2,2,Mazagao,BR,-0.1150,-51.2894,89.65,70,46,4.59,scattered clouds
8,8,Impfondo,CG,1.6381,18.0667,78.58,73,100,2.44,overcast clouds
14,14,Vila Velha,BR,-20.3297,-40.2925,75.15,76,75,18.41,broken clouds
19,19,Koungou,YT,-12.7336,45.2042,76.71,83,0,6.91,clear sky
20,20,Beausoleil,FR,43.7490,7.4170,85.23,71,20,5.75,few clouds
24,24,Victoria,HK,22.2855,114.1577,79.97,90,100,1.01,overcast clouds
26,26,Belmonte,BR,-15.8631,-38.8828,77.94,61,69,13.78,broken clouds
28,28,Atuona,PF,-9.8000,-139.0333,77.27,73,39,19.80,scattered clouds
29,29,Nishihara,JP,26.1842,127.7558,78.85,97,75,21.85,light intensity shower rain


In [13]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,
2,Mazagao,BR,89.65,scattered clouds,-0.1150,-51.2894,
8,Impfondo,CG,78.58,overcast clouds,1.6381,18.0667,
14,Vila Velha,BR,75.15,broken clouds,-20.3297,-40.2925,
19,Koungou,YT,76.71,clear sky,-12.7336,45.2042,
20,Beausoleil,FR,85.23,few clouds,43.7490,7.4170,
24,Victoria,HK,79.97,overcast clouds,22.2855,114.1577,
26,Belmonte,BR,77.94,broken clouds,-15.8631,-38.8828,
28,Atuona,PF,77.27,scattered clouds,-9.8000,-139.0333,
29,Nishihara,JP,78.85,light intensity shower rain,26.1842,127.7558,


In [19]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    Lat = row['Lat']
    Lng = row['Lng']
    params["location"] = f"{Lat},{Lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [30]:
# 7. Drop the rows where there is no Hotel Name.
hotel_df['Hotel Name'].replace('', 0, inplace=True)
clean_hotel_df = hotel_df.dropna()
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Puerto Ayora,EC,80.56,broken clouds,-0.7393,-90.3518,Finch Bay Galapagos Hotel
2,Mazagao,BR,89.65,scattered clouds,-0.1150,-51.2894,Recanto dos Ferreiras
8,Impfondo,CG,78.58,overcast clouds,1.6381,18.0667,Auberge La Place de Jules
14,Vila Velha,BR,75.15,broken clouds,-20.3297,-40.2925,Hotel Vitória Palace
19,Koungou,YT,76.71,clear sky,-12.7336,45.2042,Villa Netibor
20,Beausoleil,FR,85.23,few clouds,43.7490,7.4170,Le Méridien Beach Plaza
24,Victoria,HK,79.97,overcast clouds,22.2855,114.1577,Mini Central
26,Belmonte,BR,77.94,broken clouds,-15.8631,-38.8828,Pousada Monte Carmelo O Pão
28,Atuona,PF,77.27,scattered clouds,-9.8000,-139.0333,Villa Enata
29,Nishihara,JP,78.85,light intensity shower rain,26.1842,127.7558,Minshuku Agaihama


In [31]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [32]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and </dt><dd>{Max Temp} °F</dd>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [33]:
# 11a. Add a marker layer for each city to the map. 
locations = clean_hotel_df[["Lat", "Lng"]]
max_temp = clean_hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)


# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))